# Project
## BMES 543
### Matthew Falcione

### Adjust Notebook Settings

In [1]:
# set constants that the user can adjust to fit their system constraints and needs
# set LOCAL to False if running on Google Colab
LOCAL = True
# set SKIP_PREPROCESSING to False if you want to run all the preprocessing steps
# *WARNING* it will take over an hour to run the feature selection steps on Google Colab
SKIP_PREPROCESSING = True
# set SKIP_RUN to False if you want to run all the classification steps
# *WARNING* it will take over two hours to run the classification steps on Google Colab
SKIP_RUN = True

### Import modules

In [2]:
import os
import sys
import pickle
import zipfile
import warnings
import requests
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2, RFECV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

In [3]:
# adjust notebook display settings
%matplotlib inline
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50

# add BMES directory if on local machine
if LOCAL:
    sys.path.append(os.environ['BMESAHMETDIR'])
    import bmes
else:
    # Google Colab requires using joblib to load pickle files instead of pickle.load
    from sklearn.externals import joblib

### Download dataset from GitHub

In [4]:
# create dictionaries to house all dataset data
# these will be nested dictonaries that contain all relevant data for each respective dataset
dataset_name = ['adenocarcinoma', 'colon', 'prostate', 'lymphoma',
                'nci', 'srbct', 'leukemia', 'brain', 'breast.2.class', 'breast.3.class']
# initalize dictionaries for each dataset
dataset_dict = {dataset: {} for dataset in dataset_name}

In [5]:
# get data and class file paths
def get_data_class_file_paths(dataset_name):
    if LOCAL:
        data_file_path = f'{bmes.tempdir()}/data.sets/{dataset_name}.data.txt'
        class_file_path = f'{bmes.tempdir()}/data.sets/{dataset_name}.class.txt'
    else:
        data_file_path = f'data.sets/{dataset_name}.data.txt'
        class_file_path = f'data.sets/{dataset_name}.class.txt'
        
    return data_file_path, class_file_path

In [6]:
# retrieve file from url
def download_files(dataset_dict):
    
    # test if the data files are present
    try:
        # loop through each dataset's dict for file names
        for dataset_name in dataset_dict:
            # generate file paths for the corresponding data and class label files
            data_file_path, class_file_path = get_data_class_file_paths(dataset_name)

            # ensure that the file paths exist
            assert (os.path.exists(data_file_path) and os.path.exists(class_file_path)), 'Files are not in the data directory.'       
        
            # add file paths to respective datasets
            dataset_dict[dataset_name]['data_file_path'] = data_file_path
            dataset_dict[dataset_name]['class_file_path'] = class_file_path
            
        print('Dataset files already downloaded, skipping...')
        
        return dataset_dict
                
    except AssertionError as e: 
        print(e)
        print('Downloading datasets.')
        
        # dataset value and class download link
        GITHUB_DATASET_LINK = 'https://github.com/rdiaz02/varSelRF-suppl-mat/blob/3ef2b5156f288cdad153998084bf77578c90393d/data.sets.zip?raw=true'
        
        # generate file paths for the zipped and unzipped data and class label files
        if LOCAL:
            github_zipped_folder_path = f'{bmes.tempdir()}/data.sets.zip'
            github_unzipped_folder_path = bmes.tempdir()
        else:
            github_zipped_folder_path = f'data.sets.zip'
            github_unzipped_folder_path = ""
            
        if not os.path.exists(github_zipped_folder_path):
            # download the dataset from GitHub
            dataset_request = requests.get(GITHUB_DATASET_LINK, allow_redirects=True)
            # write folder to github_zipped_folder_path
            with open(github_zipped_folder_path, 'wb') as file:
                file.write(dataset_request.content)
            
        # extract datasets folder
        with zipfile.ZipFile(github_zipped_folder_path, "r") as zip_ref:
                zip_ref.extractall(github_unzipped_folder_path)
        
        
        # loop through each dataset's dict for file names
        for dataset_name in dataset_dict:
            # generate file paths for the corresponding data and class label files
            data_file_path, class_file_path = get_data_class_file_paths(dataset_name)

            # add file paths to respective datasets
            dataset_dict[dataset_name]['data_file_path'] = data_file_path
            dataset_dict[dataset_name]['class_file_path'] = class_file_path

    print('Done!')
        
    return dataset_dict

In [7]:
# download dataset files and class labels if running the data preprocesing steps
if not SKIP_PREPROCESSING:
    dataset_dict = download_files(dataset_dict=dataset_dict)

### Load and clean datasets

In [8]:
# convert text class labels to numeric (this is present in some of the datasets)
def clean_class_data(class_array):
    # covert text labels of 'Normal'/'Tumor' to numeric
    if 'Normal' in list(class_array):
        return np.where(class_array == 'Normal', 0, 1)
    else:
        return class_array

In [9]:
# create dictionaries of the downloaded file's class target values, attribute values, and attribute names
def load_datasets(dataset_dict):
    # X: attribute values
    # y: target values
    # names: attribute names

    # loop through each dataset
    for dataset_name in dataset_dict:
        # get file paths for data and class files
        data_file_path = dataset_dict[dataset_name]['data_file_path']
        class_file_path = dataset_dict[dataset_name]['class_file_path']
        
        df_data = pd.read_csv(data_file_path, delimiter='\t')
        # determine if the gene column was set as index
        # certain datasets used '#' to denote the index column
        if df_data.columns[0] == '#':
            df_data = pd.read_csv(data_file_path, delimiter='\t', index_col='#')
        df_class = pd.read_csv(class_file_path, delimiter='\t', header=None)

        # drop missing samples (columns)
        df_class.dropna(inplace=True, axis='columns')
        df_data.dropna(inplace=True, axis='columns')

        # append each dataset's class target values, attribute values, and attribute names to their respective dictionaries
        dataset_dict[dataset_name]['X'] = df_data.T.values
        dataset_dict[dataset_name]['y'] = clean_class_data(df_class.values[0])
        dataset_dict[dataset_name]['names'] = list(df_data.T.columns)
        
    return dataset_dict

In [10]:
# load dataset class target values, attribute values, and attribute names if running the data preprocesing steps
if not SKIP_PREPROCESSING:
    dataset_dict = load_datasets(dataset_dict)

### Create repeated stratified k-fold cross-validation generator

In [11]:
# 3-fold, 10 repeat cross-validation
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=10, random_state=0)

### Initialize classification models

In [12]:
# initiate random forest classifier model
rf_model = RandomForestClassifier(n_estimators=100, random_state=0)

# initiate XGBoost classifier model
xgboost_model = XGBClassifier(random_state=0)

# control how many hidden units are used for the network
NUM_HIDDEN = 5
# initiate neural network classifier model
nn_model = MLPClassifier(solver='adam', hidden_layer_sizes=(NUM_HIDDEN,), random_state=0, max_iter=1000)

# initiate LDA model
lda_model = LinearDiscriminantAnalysis()

# initiate KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)

# initiate logisitc regression model
lr_model = LogisticRegression(random_state=0)

# initiate SVM model
svm_model = SVC(kernel='rbf', probability=True)

### Create feature selection methods

* K-best feature selection (filter method)
* Tree-based feature selection (embedded method)
* Recursive feature elimination (hybrid method)

In [13]:
# create dictionary to house feature selection types and corresponding X dataset label
feature_selection_types = ['Raw', 'K-best', 'Tree-based', 'Random Forest RFE', 'XGBoost RFE', 'Logistic Regression RFE']
X_dataset_labels = ['X', 'X_k_best', 'X_tree_based', 'X_random_forest_rfe', 'X_xgboost_rfe', 'X_lr_rfe']
feature_selection_dict = dict(zip(feature_selection_types, X_dataset_labels))

#### K-best (univariate)

In [14]:
# if running the data preprocesing steps
if not SKIP_PREPROCESSING:
    # apply SelectKBest class to extract top 10 best features using a chi^2 distribution
    k = 10
    best_features = SelectKBest(score_func=chi2, k=k)

    # fit the feature selction to the X and y values
    for dataset_name in dataset_dict:
        # add dataset's selected feature subset to dict
        # X values must be positive
        dataset_dict[dataset_name]['X_k_best'] = best_features.fit_transform(abs(dataset_dict[dataset_name]['X']),
                                                                             dataset_dict[dataset_name]['y'])

#### Tree-based feature selection

In [15]:
# if running the data preprocesing steps
if not SKIP_PREPROCESSING:
    for dataset_name in dataset_dict:
        # fit the random forest model to X and y
        _ = rf_model.fit(dataset_dict[dataset_name]['X'], dataset_dict[dataset_name]['y'])
        # get the feature importances
        importances = rf_model.feature_importances_

        # get the top 10 most important attributes from X
        names = dataset_dict[dataset_name]['names']
        importances_series = pd.Series(importances, index=names)
        top_n_most_important_attributes = importances_series.nlargest(10).index.tolist()

        # get the column numbers of X from the top 10 most important features
        attribute_column_numbers = [names.index(attribute) for attribute in top_n_most_important_attributes]

        # extract 10 most important features from X
        dataset_dict[dataset_name]['X_tree_based']  = dataset_dict[dataset_name]['X'][:, attribute_column_numbers]

#### Recursive feature elimination

In [16]:
# if running the data preprocesing steps
if not SKIP_PREPROCESSING:
    for dataset_name in dataset_dict:
        # build the RFE with cross-validation on the random-forest model (remove 10% in each iteration -> step=0.1)
        random_forest_rfe = RFECV(rf_model, min_features_to_select=1, step=0.1, cv=cv, scoring='accuracy')
        xgboost_rfe = RFECV(xgboost_model, min_features_to_select=1, step=0.1, cv=cv, scoring='accuracy')
        lr_rfe = RFECV(lr_model, min_features_to_select=1, step=0.1, cv=cv, scoring='accuracy')

        # fit RFE to data
        rf_selector = random_forest_rfe.fit(dataset_dict[dataset_name]['X'], dataset_dict[dataset_name]['y'])
        xgboost_selector = xgboost_rfe.fit(dataset_dict[dataset_name]['X'], dataset_dict[dataset_name]['y'])
        lr_selector = lr_rfe.fit(dataset_dict[dataset_name]['X'], dataset_dict[dataset_name]['y'])

        # get boolean support values for each variable (gene)
        rf_selected_features_boolean_index = rf_selector.support_
        xgboost_selected_features_boolean_index = xgboost_selector.support_
        lr_selected_features_boolean_index = lr_selector.support_

        # select index of features that were kept by feature selection model
        rf_selected_features_index = rf_selected_features_boolean_index.nonzero()[0]
        xgboost_selected_features_index = xgboost_selected_features_boolean_index.nonzero()[0]
        lr_selected_features_index = lr_selected_features_boolean_index.nonzero()[0]

        # extract kept features from data
        dataset_dict[dataset_name]['X_random_forest_rfe']  = dataset_dict[dataset_name]['X'][:, rf_selected_features_index]
        dataset_dict[dataset_name]['X_xgboost_rfe']  = dataset_dict[dataset_name]['X'][:, xgboost_selected_features_index]
        dataset_dict[dataset_name]['X_lr_rfe']  = dataset_dict[dataset_name]['X'][:, lr_selected_features_index]

### Save Dataset with Selected Features

In [17]:
# data_file_path will use the lcoal directory if on Google Colab
# LOCAL will use bmes.tempdir()
if not LOCAL:
    data_file_path = 'dataset_dict.pickle'
else:
    data_file_path = f'{bmes.tempdir()}/dataset_dict.pickle'

In [18]:
# if running the data preprocesing steps
if not SKIP_PREPROCESSING:
    # save dictonaries with data so we don't have to run the code again
    pickle.dump(dataset_dict, open(data_file_path, "wb"))

### Load Selected Features Dataset

In [19]:
# if preprocessed data file does not exist
if not os.path.exists(data_file_path):
    # if not running the data preprocesing steps
    if SKIP_PREPROCESSING: 
        # initiate dataset_dict link
        dataset_dict_link = 'https://drexel0-my.sharepoint.com/:u:/g/personal/mjf378_drexel_edu/ER9Yrk5KV6xJvYqvCO975TcBFESzfq8fElv346c9-dD_GQ?download=1'
        dataset_dict_request = requests.get(dataset_dict_link, allow_redirects=True)

        # write files to bmes.datadir() or local directory with respective names
        with open(data_file_path, 'wb') as file:
            file.write(dataset_dict_request.content)

# load dataset_dict from pickle file
if LOCAL:
    dataset_dict = pickle.load(open(data_file_path, 'rb'))
else:
    # Google Colab requires using joblib to load pickle files instead of pickle.load
    dataset_dict = joblib.load(data_file_path)

### Create classification evaluation metric function

In [20]:
def classification_evaluation_metrics(model, X, y, cv, classification_type=None, feature_selection_type=None, dataset_name=None):
    # calculate accuracy value from model
    # in multi-class case (where only one possible target) - micro-precision=micro-recall=micro-F1=accuracy
    # micro: calculate metrics globally by considering each element of the label indicator matrix as a label
    accuracy_scores = cross_val_score(model, X, y, scoring='f1_micro', cv=cv, error_score='raise')
    accuracy = round(np.median(accuracy_scores), 3)
    
    # calculate AUC-ROC value from model
    # ovo: stands for one-vs-one, computes the average AUC of all possible pairwise combinations of classes
    auc_roc_scores = cross_val_score(model, X, y, scoring='roc_auc_ovo_weighted', cv=cv, error_score='raise')
    auc_roc = round(np.median(auc_roc_scores), 3)

    return accuracy, auc_roc

### Run classification algorithms

In [21]:
# create list of columns for the model output dataframe
column_list = ['Dataset Name','Classification Type'] + feature_selection_types
# initialize dict of classification models and classification type descriptions
model_list = [rf_model, xgboost_model, nn_model, lda_model, knn_model, lr_model, svm_model]
classification_type_list = ['Random Forest', 'XGBoost', 'Neural Network', 'LDA', 'KNN', ' Logistic Regression', 'SVM']
model_classification_dict = dict(zip(model_list, classification_type_list))

In [22]:
def get_output_file_path(suffix):
    # output_file_path will use the local directory if on Google Colab
    # LOCAL will use bmes.tempdir()
    if not LOCAL:
        accuracy_output_file_path = f'accuracy_list_{suffix}.pickle'
        auc_roc_output_file_path = f'auc_roc_list_{suffix}.pickle'
    else:
        accuracy_output_file_path = f'{bmes.tempdir()}/accuracy_list_{suffix}.pickle'
        auc_roc_output_file_path = f'{bmes.tempdir()}/auc_roc_list_{suffix}.pickle'
        
    return accuracy_output_file_path, auc_roc_output_file_path

In [23]:
def export_results(accuracy_list, auc_roc_list, suffix):
    # get file path for output files
    accuracy_output_file_path, auc_roc_output_file_path = get_output_file_path(suffix)
        
    # save lists with the output so we don't have to run the code again
    pickle.dump(accuracy_list, open(accuracy_output_file_path, "wb"))
    pickle.dump(auc_roc_list, open(auc_roc_output_file_path, "wb"))     

In [24]:
def run_classification_models(cv, column_list, dataset_dict, feature_selection_dict, model_classification_dict):
    # create lists to contain the model results for each dataset and feature selection method
    # lists will contain dicts which will each be a row in the final corresponding dataframe
    accuracy_list = []
    auc_roc_list = []
    
    # loop thorough each classification model
    for model, classification_type in model_classification_dict.items():
        print(classification_type)
        print('-------------------')
         
        # loop through each dataset
        for dataset_name in dataset_dict:
            # accuracy
            # create dict that will contain model accuracy for each feature selection type
            model_accuracy_output_dict = {col: None for col in column_list}
            # add dataset name and classification type to dict
            model_accuracy_output_dict['Dataset Name'] = dataset_name
            model_accuracy_output_dict['Classification Type'] = classification_type

            # AUC-ROC
            # create dict that will contain model AUC-ROC for each feature selection type
            model_auc_roc_output_dict = {col: None for col in column_list}
            # add dataset name and classification type to dict
            model_auc_roc_output_dict['Dataset Name'] = dataset_name
            model_auc_roc_output_dict['Classification Type'] = classification_type
            
            # run classification model with each feature selection technique and dataset
            for feature_selection_type, X_dataset_label in feature_selection_dict.items(): 
                print(f'{dataset_name} - {feature_selection_type}')

                # output accauracy and aur-roc for the combinations of each model and feature selection technique
                accuracy, auc_roc = classification_evaluation_metrics(model=model, X=dataset_dict[dataset_name][X_dataset_label],
                                                                      y=dataset_dict[dataset_name]['y'], cv=cv,
                                                                      classification_type=classification_type,
                                                                      feature_selection_type=feature_selection_type,
                                                                      dataset_name=dataset_name)
            
                # add each feature selection technique output to output dict for the respective dataset
                model_accuracy_output_dict[feature_selection_type] = accuracy
                model_auc_roc_output_dict[feature_selection_type] = auc_roc
                
            # append model output dict for specific dataset to list of outputs
            accuracy_list.append(model_accuracy_output_dict)
            auc_roc_list.append(model_auc_roc_output_dict)
            
        # export results after each model in case there are any issues with the time to run the models
        export_results(accuracy_list, auc_roc_list, suffix='intermediate_output')
        print('')
    
    # save final output - this is here so that if run_classification_models is run it will not overwrite the finalized output with the intermediate output 
    export_results(accuracy_list, auc_roc_list, suffix='final_output')        
    
    return accuracy_list, auc_roc_list

In [25]:
# run all classification models
if not SKIP_RUN:
    accuracy_list, auc_roc_list = run_classification_models(cv=cv, column_list=column_list, dataset_dict=dataset_dict,
                                                          feature_selection_dict=feature_selection_dict, model_classification_dict=model_classification_dict)
else:
    # get file path for output files
    accuracy_output_file_path, auc_roc_output_file_path = get_output_file_path(suffix='final_output')
    # load dataset_dict from pickle file
    if LOCAL:
        accuracy_list = pickle.load(open(accuracy_output_file_path, 'rb'))
        auc_roc_list = pickle.load(open(auc_roc_output_file_path, 'rb'))
    else:
        # Google Colab requires using joblib to load pickle files instead of pickle.load
        accuracy_list = joblib.load(accuracy_output_file_path)
        auc_roc_list = joblib.load(auc_roc_output_file_path)

In [26]:
# main index is the dataset names
# second level index is the classification type
# columns are the feature selection methods

# values in the columns are the accuracy values
df_accuracy = pd.DataFrame(accuracy_list)
df_accuracy.set_index(['Dataset Name', 'Classification Type'], inplace=True, drop=True)
# get error rates (1 - accuracy) to directly compare to paper
df_error_rates = (1 - df_accuracy) * 100

# values in the columns are the auc-roc values
df_auc_roc = pd.DataFrame(auc_roc_list)
df_auc_roc.set_index(['Dataset Name', 'Classification Type'], inplace=True, drop=True)

### Displaying Results
Since only accuracy/error rates were provided by the paper, they are the only metric that will be analyzed here. Further work could be done to compare AUC-ROC score outputs along with supplementing the datasets with simulated data.

#### Pull results from paper

In [27]:
# "no info" denotes the minimal error we can make if we use no information from the genes
# (i.e., we always bet on the most frequent class)
no_info_error_rates_dict = {'adenocarcinoma': 0.158, 'colon': 0.355, 'prostate': 0.490,
                            'lymphoma': 0.323, 'nci': 0.852, 'srbct': 0.635, 'leukemia': 0.289,
                            'brain': 0.762, 'breast.2.class': 0.429, 'breast.3.class': 0.537}

# dict of which classification method performed best for each dataset
best_classification_method_dict = {'adenocarcinoma': 'Random Forest', 'colon': 'Srunken Centroids', 'prostate': 'Random Forest',
                                   'lymphoma': 'Random Forest', 'nci': 'Nearest Neighbors', 'srbct': 'DLDA', 'leukemia': 'SVM',
                                   'brain': 'SVM', 'breast.2.class': 'Srunken Centroids', 'breast.3.class': 'Random Forest'}

# dict of the error rate for the classification that performed best for each dataset
best_method_error_rates_dict = {'adenocarcinoma': 0.125, 'colon': 0.122, 'prostate': 0.061,
                                'lymphoma': 0.009, 'nci': 0.237, 'srbct': 0.011, 'leukemia': 0.014,
                                'brain': 0.138, 'breast.2.class': 0.324, 'breast.3.class': 0.346}

#### Calculate average performance of feature selection techniques

In [28]:
# calculate mean and std of performance of each feature selection technique across and datasets and models
feature_selection_error_rates_mean = round(df_error_rates.mean(), 3)
feature_selection_error_rates_std = round(df_error_rates.std(), 3)

# display the average performance of feature selection techniques
formatted_average_output_list = []
print('Average error rates of feature selection techniques')
print('---------------------------------------------------')
for feature_selection_type in feature_selection_types:
    formatted_average_output = f'{feature_selection_error_rates_mean[feature_selection_type]}% +/- {feature_selection_error_rates_std[feature_selection_type]}%'
    formatted_average_output_list.append(formatted_average_output)
    print(f'{feature_selection_type}: {formatted_average_output}')

Average error rates of feature selection techniques
---------------------------------------------------
Raw: 21.1% +/- 14.995%
K-best: 19.666% +/- 15.23%
Tree-based: 17.514% +/- 12.51%
Random Forest RFE: 20.456% +/- 15.26%
XGBoost RFE: 17.557% +/- 14.461%
Logistic Regression RFE: 15.623% +/- 15.517%


**Figure 1.** Average error rates of feature selection techniques. Tree-based and Logistic Regression recursive feature elimination methods performed best with percent errors of 17.514% +/- 12.51% and 15.623% +/- 15.517%, respectively, but the standard deviations are substantial compared to the averages for all methods since this mertric was calculated across multiple datasets and models. 

In [29]:
# calculate average performance of each feature selection technique for each dataset (across all models)
feature_selection_performance_dict = {}
for dataset_name in dataset_dict:
    df_feature_selection_mean = round(df_error_rates.loc[dataset_name].mean(), 3)
    df_feature_selection_std = round(df_error_rates.loc[dataset_name].std(), 3)
    
    # create list of formatted mean std output
    formatted_feature_mean_std = [f'{df_feature_selection_mean[feature]} +/- {df_feature_selection_std[feature]}' for feature in df_feature_selection_mean.index]
    
    # store each mean and std in a dictonary to convert into a dataframe
    feature_selection_performance_dict[dataset_name] = formatted_feature_mean_std
    
# display dataframe of average performance of each feature selection technique for each dataset (across all models)
df_features_mean_std = pd.DataFrame(feature_selection_performance_dict).T
df_features_mean_std.columns = feature_selection_types
# add average error rates for each feature selection technique
df_features_mean_std.loc['Average'] = formatted_average_output_list
df_features_mean_std

,Raw,K-best,Tree-based,Random Forest RFE,XGBoost RFE,Logistic Regression RFE
adenocarcinoma,15.386 +/- 1.497,17.071 +/- 4.834,14.343 +/- 6.05,15.914 +/- 0.146,23.343 +/- 21.506,15.014 +/- 1.573
colon,15.686 +/- 2.267,12.371 +/- 2.048,11.314 +/- 2.238,17.114 +/- 5.348,11.729 +/- 2.048,9.014 +/- 3.294
prostate,12.386 +/- 2.929,9.229 +/- 1.134,7.143 +/- 1.55,8.814 +/- 1.703,10.7 +/- 3.85,4.814 +/- 2.214
lymphoma,3.457 +/- 5.41,5.157 +/- 2.12,7.243 +/- 2.267,2.143 +/- 3.934,0.686 +/- 1.814,1.429 +/- 3.78
nci,39.0 +/- 10.664,48.8 +/- 5.913,41.357 +/- 8.089,39.0 +/- 10.664,31.786 +/- 7.796,39.0 +/- 10.664
srbct,6.814 +/- 5.571,6.143 +/- 2.293,13.957 +/- 0.907,5.8 +/- 5.813,1.371 +/- 2.342,1.357 +/- 3.591
leukemia,7.971 +/- 6.567,7.786 +/- 4.451,1.1 +/- 2.91,7.971 +/- 6.567,6.6 +/- 5.314,2.2 +/- 5.821
brain,31.643 +/- 9.768,19.886 +/- 4.036,22.957 +/- 7.095,30.1 +/- 9.186,26.029 +/- 9.827,13.257 +/- 9.622
breast_2_class,37.429 +/- 1.782,26.657 +/- 3.3,23.686 +/- 0.865,36.371 +/- 4.235,30.486 +/- 3.051,29.457 +/- 3.033
breast_3_class,41.229 +/- 1.928,43.557 +/- 3.611,32.043 +/- 1.441,41.329 +/- 2.341,32.843 +/- 4.561,40.686 +/- 1.897


**Table 1:** Average performance (% error) of each feature selection technique for each dataset (across all models).

In [30]:
# calculate performance of each feature selection technique for all datasets for SVM and KNN
models_to_compare = ['SVM', 'KNN']
df_models_to_compare_list = []

# loop through each model that we want to compare the performance of the raw data vs the feature selected data
for model in models_to_compare:
    df_model = df_error_rates.xs(model, level=1, drop_level=True)
    df_model[f'{model} - Raw'] = df_model['Raw']
    df_model[f'{model} - Feature Selection'] = df_model.min(axis=1)
    df_model[f'{model} - Performance Increase'] = df_model[f'{model} - Raw'] - df_model[f'{model} - Feature Selection']
    df_model.drop(columns=feature_selection_types, inplace=True)
    df_models_to_compare_list.append(df_model)

# concat the dataframe for each respective technique and dispay it
df_combined_models_to_compare = pd.concat(df_models_to_compare_list, axis=1)
average_performance_formatted = [f'{df_combined_models_to_compare[column].mean():0.3} +/- {df_combined_models_to_compare[column].std():0.3}' for column in df_combined_models_to_compare.columns]
df_combined_models_to_compare.loc['Average'] = average_performance_formatted
df_combined_models_to_compare

,SVM - Raw,SVM - Feature Selection,SVM - Performance Increase,KNN - Raw,KNN - Feature Selection,KNN - Performance Increase
Dataset Name,,,,,,
adenocarcinoma,16.0,16.0,0.0,16.0,16.0,0.0
colon,14.3,9.5,4.8,19.0,9.5,9.5
prostate,14.7,4.4,10.3,14.7,5.9,8.8
lymphoma,0.0,0.0,0.0,0.0,0.0,0.0
nci,35.0,30.0,5.0,35.0,33.3,1.7
srbct,2.4,0.0,2.4,9.5,0.0,9.5
leukemia,7.7,0.0,7.7,0.0,0.0,0.0
brain,35.7,7.1,28.6,28.6,14.3,14.3
breast_2_class,40.0,23.1,16.9,38.5,25.5,13.0


**Table 2:** Performance (% error) of raw data vs feature-selected data with SVM and KNN models for all datasets. Feature selection performance increases across all datasets for SVM ranged from 0% to 28.6% with an average of 8.47% +/- 8.73%, and performance increases across all datasets for KNN ranged from 0% to 14.3% with an average of 6.88% +/- 5.82%.

#### Compare top performing methods to those from the paper

In [31]:
# store best performance dataframes for each classification model for each dataset
model_performance_list = []

# find best performance for each classification model for each dataset
for classification_type in classification_type_list:
    df_model = df_error_rates.xs(classification_type, level=1, drop_level=False)
    df_model = df_model.droplevel('Classification Type')
    df_best_performance = pd.DataFrame(round(df_model.min(axis=1), 3), columns=[classification_type])
    model_performance_list.append(df_best_performance)
    
# concat all best performance dataframes across all datasets
df_best_overall_performance = pd.concat(model_performance_list, axis=1)
# "no info" denotes the minimal error we can make if we use no information from the genes
# (i.e., we always bet on the most frequent class)
df_best_overall_performance.insert(0, 'No Info', no_info_error_rates_dict.values())
df_best_overall_performance['No Info'] = df_best_overall_performance['No Info'] * 100
best_performance_average_formatted = [f'{df_best_overall_performance[column].mean():0.3} +/- {df_best_overall_performance[column].std():0.3}' for column in df_best_overall_performance.columns]
df_best_overall_performance_with_average = df_best_overall_performance.copy()
df_best_overall_performance_with_average.loc['Average'] = best_performance_average_formatted

# display best performance dataframes across all datasets for each classification method
df_best_overall_performance_with_average

,No Info,Random Forest,XGBoost,Neural Network,LDA,KNN,Logistic Regression,SVM
Dataset Name,,,,,,,,
adenocarcinoma,15.8,8.0,8.0,16.0,11.5,16.0,12.0,16.0
colon,35.5,10.0,14.3,5.0,9.5,9.5,5.0,9.5
prostate,49.0,5.9,8.8,2.9,2.9,5.9,2.9,4.4
lymphoma,32.3,0.0,4.8,0.0,0.0,0.0,0.0,0.0
nci,85.2,25.0,39.0,40.0,33.3,33.3,19.5,30.0
srbct,63.5,0.0,4.8,0.0,0.0,0.0,0.0,0.0
leukemia,28.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
brain,76.2,14.3,14.3,21.4,7.1,14.3,0.0,7.1
breast_2_class,42.9,23.1,23.5,23.5,23.1,25.5,24.0,23.1


**Table 3:** Performance (% error) of the best feature selection method for each classification type across each dataset. Each model performed substantially better than the `No Info` model in the first column, which is what would be expected from a random guess based on always betting on the most frequent class. Logistic regression narrowly beat Random Forest for the best performing model with an average error of 9.46% +/- 11.6%. However, these values could be skewed by model overfitting.

In [32]:
# get best model performance from each of our models
best_models = df_best_overall_performance.idxmin(axis=1).to_dict()
best_performance = round(df_best_overall_performance.min(axis=1), 3)

df_compare_model_performance = pd.DataFrame.from_dict(best_models, orient='index', columns=['Our Best Model'])
df_compare_model_performance['Best From Our Models'] = best_performance

# compare our best model to the best models from the paper
df_compare_model_performance['Best Method From Paper'] = best_classification_method_dict.values()
df_compare_model_performance['Best From Paper'] = best_method_error_rates_dict.values()
df_compare_model_performance['Best From Paper'] = df_compare_model_performance['Best From Paper'] * 100
df_compare_model_performance.insert(0, 'No Info', no_info_error_rates_dict.values())
df_compare_model_performance['No Info'] = df_compare_model_performance['No Info'] * 100

df_compare_model_performance.index.name = 'Dataset Name'
df_compare_model_performance

,No Info,Our Best Model,Best From Our Models,Best Method From Paper,Best From Paper
Dataset Name,,,,,
adenocarcinoma,15.8,Random Forest,8.0,Random Forest,12.5
colon,35.5,Neural Network,5.0,Srunken Centroids,12.2
prostate,49.0,Neural Network,2.9,Random Forest,6.1
lymphoma,32.3,Random Forest,0.0,Random Forest,0.9
nci,85.2,Logistic Regression,19.5,Nearest Neighbors,23.7
srbct,63.5,Random Forest,0.0,DLDA,1.1
leukemia,28.9,Random Forest,0.0,SVM,1.4
brain,76.2,Logistic Regression,0.0,SVM,13.8
breast_2_class,42.9,Random Forest,23.1,Srunken Centroids,32.4


**Table 4:** Performance comparison (% error) of our best methods to those from the paper. Random Forest was most often the best model choice from both the paper and our models. The table also shows that our models were able to perform better than those from the paper, likely due to the additional feature selection techniques used.